<a href="https://colab.research.google.com/github/charoo-rumsan/community_tool_R-D/blob/main/Phone_no_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install --quiet polars phonenumbers gspread google-auth


In [ ]:
 '''
from google.colab import auth
auth.authenticate_user()   # Follow the OAuth flow

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Provide your sheet URL or key here:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1APm4yeggrP75hAlJdkQM4D4c0sIEWGzYG5Sdm8qwUwI/edit?gid=1461989259#gid=1461989259"
sh = gc.open_by_url(SHEET_URL)    # or: gc.open_by_key("SHEET_ID")
worksheet = sh.sheet1  # or sh.worksheet("Sheet name")

rows = worksheet.get_all_values()
if len(rows) < 1:
    raise RuntimeError("Sheet appears empty.")
header = rows[0]
data = rows[1:]  '''


'\nfrom google.colab import auth\nauth.authenticate_user()   # Follow the OAuth flow\n\nimport gspread\nfrom google.auth import default\ncreds, _ = default()\ngc = gspread.authorize(creds)\n\n# Provide your sheet URL or key here:\nSHEET_URL = "https://docs.google.com/spreadsheets/d/1APm4yeggrP75hAlJdkQM4D4c0sIEWGzYG5Sdm8qwUwI/edit?gid=1461989259#gid=1461989259"\nsh = gc.open_by_url(SHEET_URL)    # or: gc.open_by_key("SHEET_ID")\nworksheet = sh.sheet1  # or sh.worksheet("Sheet name")\n\nrows = worksheet.get_all_values()\nif len(rows) < 1:\n   raise RuntimeError("Sheet appears empty.")\nheader = rows[0]\ndata = rows[1:]  '

PROTOTYPE

In [ ]:
import phonenumbers
from phonenumbers import geocoder, NumberParseException

 Classify and validate a phone number.
    
    Args:
        phone_str (str): Raw phone number string
        default_region (str): Country code to assume if no '+' prefix is given (default: Nepal 'NP')
    
    Returns:
        dict: { 'phone': str, 'valid': bool, 'country': str }

In [ ]:
def classify_number(phone_str, default_region="NP"):

    result = {"phone": phone_str, "valid": False, "country": None}

    if not phone_str or str(phone_str).strip() == "":
        return result

    try:
        # Parse number
        num = phonenumbers.parse(str(phone_str), default_region)

        # Check validity
        if phonenumbers.is_valid_number(num):
            result["valid"] = True
            result["country"] = geocoder.description_for_number(num, "en")
        else:
            result["valid"] = False
    except NumberParseException:
        result["valid"] = False

    return result

In [ ]:
examples = [
    "+9779812345678",  # Valid Nepali
    "9812345678",      # Nepali without +
    "+14155552671",    # US number
    "12345",           # Invalid
]
for ex in examples:
    print(classify_number(ex))

IMPLEMENTATION

In [ ]:
!pip install --quiet polars phonenumbers



In [ ]:
import polars as pl

# Adjust file path if needed (UPLOAD CSV FILE)
df = pl.read_csv("/content/first_100_rows (1) - first_100_rows (1).csv.csv")
print(df.shape)
print(df.head())


In [ ]:
import re
import polars as pl

candidates = []
for col in df.columns:
 sample = df[col].drop_nulls().cast(pl.Utf8).head(20).to_list()
 if any(re.search(r"\d{7,}", x) for x in sample):  # look for 7+ digits
     candidates.append(col)

candidates

Google’s libphonenumber (and its ports) already encode these rules → it can:

Parse numbers,

Check validity (length/prefix match official plan),

Tell you the country, carrier, and type (mobile/landline/etc.

In [ ]:
#Validation + Country function
import phonenumbers
from phonenumbers import geocoder

def check_number(phone_str, default_region="NP"):
    try:
        num = phonenumbers.parse(str(phone_str), default_region)
        if not phonenumbers.is_valid_number(num):
            return {"phone": phone_str, "valid": False, "country": None}
        country = geocoder.description_for_number(num, "en")
        return {"phone": phone_str, "valid": True, "country": country}
    except Exception:
        return {"phone": phone_str, "valid": False, "country": None}


In [ ]:
phone_col = "Phone number (फोन नं)"  # <-- change to your actual column name

results = [check_number(p) for p in df[phone_col].to_list()]
res_df = pl.DataFrame(results)

final_df = res_df
final_df.head(10)


In [ ]:
out_path = "/content/validated_numbers.csv"
final_df.write_csv(out_path)

import pandas as pd
from google.colab import files
files.download(out_path)


Polars + TensorFlow

In [ ]:
!pip install polars tensorflow


In [ ]:
import polars as pl

# Load your uploaded dataset (adjust path if needed)
df = pl.read_csv("/content/first_100_rows (1) - first_100_rows (1).csv.csv")  # update file name

# Preview
print(df.head())

print(df.columns)

Preprocessing

In [ ]:
import numpy as np
import string

# Extract numbers as a list
numbers = df["phonenumber"].fill_null("").to_list()
print(numbers)

# Character mapping
chars = list("+0123456789")
char_to_idx = {c: i+1 for i,c in enumerate(chars)}  # reserve 0 for padding
maxlen = 15  # max sequence length

def encode(s):
    seq = [char_to_idx.get(c, 0) for c in str(s)]
    return seq[:maxlen] + [0]*(maxlen - len(seq))

X_enc = np.array([encode(n) for n in numbers])


In [ ]:

import numpy as np
import string

# Extract numbers as a list
numbers = df["Phone number (फोन नं)"].fill_null("").to_list()
print(numbers)

# Character mapping
chars = list("+0123456789")
char_to_idx = {c: i+1 for i,c in enumerate(chars)}  # reserve 0 for padding
maxlen = 15  # max sequence length

def encode(s):
    seq = [char_to_idx.get(c, 0) for c in str(s)]
    return seq[:maxlen] + [0]*(maxlen - len(seq))

X_enc = np.array([encode(n) for n in numbers])

In [ ]:
import random
from tensorflow import keras
from tensorflow.keras import layers

# Generate synthetic training data
def generate_number(nepali=True, valid=True):
    if not valid:
        return "".join(random.choices(string.digits, k=random.randint(3, 5)))
    if nepali:
        return "+97798" + "".join(random.choices(string.digits, k=8))
    else:
        prefixes = ["+1", "+91", "+44", "+81"]
        return random.choice(prefixes) + "".join(random.choices(string.digits, k=8))

X_train, y_train = [], []
for _ in range(3000):
    X_train.append(encode(generate_number(True, True))); y_train.append(1)
    X_train.append(encode(generate_number(False, True))); y_train.append(2)
    X_train.append(encode(generate_number(True, False))); y_train.append(0)
    X_train.append(encode(generate_number(False, False))); y_train.append(0)

X_train = np.array(X_train)
y_train = keras.utils.to_categorical(y_train, num_classes=3)

# Model
model = keras.Sequential([
    layers.Embedding(input_dim=len(char_to_idx)+1, output_dim=8, input_length=maxlen),
    layers.LSTM(32),
    layers.Dense(32, activation="relu"),
    layers.Dense(3, activation="softmax")
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train quickly
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)


In [ ]:
preds = model.predict(X_enc, verbose=0)
labels = ["Invalid", "Nepali", "International"]

results = [labels[np.argmax(p)] for p in preds]

# Add back into Polars DataFrame

df_out = pl.DataFrame({
    "Phone number (फोन नं)": df["Phone number (फोन नं)"],
    "Classification": results
})


df_out.head(10)


In [ ]:
df_out.write_csv("/content/phone_classification_results.csv")
